# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [5]:
initdate = datetime(2020,5,15)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. Meanwhile we will adjust this data for regions, then I'll add Heath sector as the minimum territorial unit for SEIRHVD models.

In [6]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing Sochimi Data
Importing Accumulated Deaths
Importing Active Infected by Minciencia
Done


## Simulation Parameters

In [7]:
# Simulation parameters
tsim = 1000
beta = 0.2
mu = 0.8
k = 0.1

SeroPrevFactor=0.05 # SeroPrevalence Factor (The proportion of the population that participates in the dynamics)
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [111]:
# Quarantines 
max_mob1 = 0.55
max_mob2 = 0.75
max_mob3 = 0.95
rem_mob = 0.45
t_end_date=datetime(2020,8,25)
t_end=(t_end_date-initdate).days
#alpha = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once').alpha(t)
alpha1 = Quarantine(rem_mob,max_mob1,qp=0,iqt=0,fqt=t_end).alpha
alpha2 = Quarantine(rem_mob,max_mob2,qp=0,iqt=0,fqt=t_end).alpha
alpha3 = Quarantine(rem_mob,max_mob3,qp=0,iqt=0,fqt=t_end).alpha

## Underreport
Fraction of Infected detected/reported  
    - Imi_det: fraction of Mild detected
    - Imi_as: fraction of asymptomatic detected
    
If both are 1, means that all infected are detected so I_det = I

In [112]:
Imi_det = 0.56
Ias_det = 0

## Simulation
Initialize Simulation Object, and run the simulation. 

In [113]:
simulation1 = SEIRHVD(tsim,beta,mu,alpha1,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM,Imi_det = Imi_det,Ias_det = Ias_det)
simulation2 = SEIRHVD(tsim,beta,mu,alpha2,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM,Imi_det = Imi_det,Ias_det = Ias_det)
simulation3 = SEIRHVD(tsim,beta,mu,alpha3,k=k,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,InitialConditions = RM,Imi_det = Imi_det,Ias_det = Ias_det)


InitialCondition Object Data
InitialCondition Object Data
InitialCondition Object Data


## Modify State Parameters:
        self.pE_Ias = 0.4# Transicion de Expuesto a Infectado asintomatico
        self.tE_Ias = 5.0

        self.pE_Imi = 0.55#78 # Transicion de Expuesto a Infectado Mild
        self.tE_Imi = 5.0

        self.pE_Icr = 0.01666#8 # Transicion de Expuesto a Infectado critico
        self.tE_Icr = 3.0

        self.pE_Ise = 0.03334#14 # Transicion de Expuesto a Infectado Severo
        self.tE_Ise = 3.0

        self.pIas_R = 1.0   # Transicion de Infectado asintomatico a Recuperado
        self.tIas_R = 10.0 

        self.pImi_R = 1.0  # Transicion de Infectado mild a Recuperado
        self.tImi_R = 15.0

        self.pIse_Hse = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
        self.tIse_Hse = 3.0 

        self.pIse_D = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
        self.tIse_D = 3.0         

        self.pIcr_V = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
        self.tIcr_V = 3.0 

        self.pIcr_D = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
        self.tIcr_D = 3.0         

        self.pHse_R = 0.97 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
        self.tHse_R = 11.0

        self.pHse_V = 0.03 # Transicion de Hospitalizado a VMI (si no ha colapsado VMI)
        self.tHse_V = 3.0

        self.pHse_D = 0.03 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
        self.tHse_D = 3.0        

        self.pV_Hout = 0.5 # Mejora de ventilado hospitalizado, transita a Hout
        self.tV_Hout = 15.0

        self.pV_D = 0.5 # VMI to Death
        self.tV_D = 15.0

        self.pHout_R = 1.0 # Mejora del paciente hospitalizado, Hout a R
        self.tHout_R = 4.0

        self.pD_B = 1.0 # Entierros
        self.tD_B = 1.0 

        self.betaD = 0
        self.eta = 0.0 # tasa de perdida de inmunidad (1/periodo)

In [114]:
pE_Ias = 0.43
pE_Imi = 0.55
pE_Ise = 0.015
pE_Icr = 0.005

# Function for setting new Params:
simulation.setnewparams()

In [115]:
simulation1.pE_Ias = pE_Ias
simulation1.pE_Imi = pE_Imi
simulation1.pE_Ise = pE_Ise
simulation1.pE_Icr = pE_Icr

simulation2.pE_Ias = pE_Ias
simulation2.pE_Imi = pE_Imi
simulation2.pE_Ise = pE_Ise
simulation2.pE_Icr = pE_Icr

simulation3.pE_Ias = pE_Ias
simulation3.pE_Imi = pE_Imi
simulation3.pE_Ise = pE_Ise
simulation3.pE_Icr = pE_Icr

## Run simulation

In [116]:
simulation1.integr_sci(0,tsim,0.1)
simulation2.integr_sci(0,tsim,0.1)
simulation3.integr_sci(0,tsim,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 2207
     njev: 33
      nlu: 33
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.07133033e-08, 2.14266065e-08, 1.07154459e-04,
       2.14287492e-04, 3.21420524e-04, 1.39275085e-03, 2.46408118e-03,
       3.53541150e-03, 4.60674183e-03, 1.53200451e-02, 2.60333484e-02,
       3.67466516e-02, 4.74599549e-02, 5.81732581e-02, 1.65306291e-01,
       2.72439323e-01, 3.79572356e-01, 4.86705389e-01, 5.93838421e-01,
       7.00971454e-01, 1.04981230e+00, 1.39865315e+00, 1.74749400e+00,
       2.09633484e+00, 2.44517569e+00, 2.89204236e+00, 3.33890903e+00,
       3.78577570e+00, 3.81370487e+00, 3.84163403e+00, 3.84721987e+00,
       3.85280570e+00, 3.85839153e+00, 3.86660209e+00, 3.87178753e+00,
       3.87576078e+00, 3.87896119e+00, 3.88216160e+00, 3.88443628e+00,
       3.88671096e+00, 3.88854399e+00, 3.89037703e+00, 3.89221006e+00,
       3.89443290e+00, 3.89665573e+00

# Analysis - Detected infected

The following plots will show the simulation results with the real data

In [117]:
# Days to plot (Today + ..)
days = currentday+130
index1 = np.searchsorted(simulation1.t,days)
index2 = np.searchsorted(simulation2.t,days)
index3 = np.searchsorted(simulation3.t,days)

### Errors

In [118]:
simulation = simulation1
t_end_idx = np.where(np.array(RM.tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t_end_idx])
E_I = np.sum(abs(RM.Ir[:t_end_idx]-simulation.I_det[idx]))/(np.mean(RM.Ir[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_d_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_d_r_tr[:t_end_idx])
E_Id = np.sum(abs(RM.I_d_r[:t_end_idx]-simulation.I_d_det[idx]))/(np.mean(RM.I_d_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_ac_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_ac_r_tr[:t_end_idx])
E_Iac = np.sum(abs(RM.I_ac_r[:t_end_idx]-simulation.I_ac_det[idx]))/(np.mean(RM.I_ac_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.Br_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.Br_tr[:t_end_idx])
E_D = np.sum(abs(RM.Br[:t_end_idx]-simulation.B[idx]))/(np.mean(RM.Br[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_UCI = np.sum(abs(RM.Hr[:t_end_idx]-(simulation.Hse[idx]+simulation.Hout[idx])))/(np.mean(RM.Hr[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_VMI = np.sum(abs(RM.Vr[:t_end_idx]-simulation.V[idx]))/(np.mean(RM.Vr[:t_end_idx])*t_end_idx)

Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}


In [119]:
Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}
print(Err)

{'Active Infected': 0.2673127045289378, 'Daily Infected': 0.48654612111901757, 'Accumulated Infected': 0.44619732830056397, 'Deaths': 0.5927969578818802, 'UCI/UTI': nan, 'VMI': 0.33150446880868517}


### Plot Grid

In [120]:


fig, axs = plt.subplots(3, 2)
axs[0,0].set_xlim(0,days)
axs[0,0].scatter(RM.tr,RM.Ir,label='Real Active Data')
axs[0,0].plot(simulation1.t[:index1],simulation1.I_det[:index1],label='Infected')
axs[0,0].plot(simulation2.t[:index2],simulation2.I_det[:index2],label='Infected')
axs[0,0].plot(simulation3.t[:index3],simulation3.I_det[:index3],label='Infected')
axs[0,0].set_title('Detectable Active Infected')
axs[0,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[0, 1].set_xlim(0,days)
axs[0, 1].plot(simulation1.t[:index1],simulation1.I_d_det[:index1],label='Daily new Infected')
axs[0, 1].plot(simulation2.t[:index2],simulation2.I_d_det[:index2],label='Daily new Infected')
axs[0, 1].plot(simulation3.t[:index3],simulation3.I_d_det[:index3],label='Daily new Infected')
axs[0, 1].scatter(RM.I_d_r_tr,RM.I_d_r,label='Daily Real Data')
#axs[0, 1].legend(loc=0)
axs[0, 1].set_title('Detected Daily infected')
axs[0,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,0].set_xlim(0,days)
axs[1,0].plot(simulation1.t[:index1],simulation1.I_ac_det[:index1],label='Detected Acummulated Infected')
axs[1,0].plot(simulation2.t[:index2],simulation2.I_ac_det[:index2],label='Detected Acummulated Infected')
axs[1,0].plot(simulation3.t[:index3],simulation3.I_ac_det[:index3],label='Detected Acummulated Infected')
axs[1,0].scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Accumulated Real Data')
axs[1,0].set_title('Detected Accumulated Infected')
axs[1,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[1,1].set_xlim(0,days)
axs[1,1].plot(simulation1.t[:index1],simulation1.B[:index1],label='Total Acummulated Deaths')
axs[1,1].plot(simulation2.t[:index2],simulation2.B[:index2],label='Total Acummulated Deaths')
axs[1,1].plot(simulation3.t[:index3],simulation3.B[:index3],label='Total Acummulated Deaths')
axs[1,1].scatter(RM.Br_tr,RM.Br,label='Real Data')
axs[1,1].set_title('Acmulated Deaths')
axs[1,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[2,0].set_xlim(0,days)
axs[2,0].plot(simulation1.t[:index1],simulation1.Hse[:index1]+simulation1.Hout[:index1],label='UCI/UTI Beds')
axs[2,0].plot(simulation2.t[:index2],simulation2.Hse[:index2]+simulation2.Hout[:index2],label='UCI/UTI Beds')
axs[2,0].plot(simulation3.t[:index3],simulation3.Hse[:index3]+simulation3.Hout[:index3],label='UCI/UTI Beds')
axs[2,0].scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
axs[2,0].scatter(RM.sochimi_tr,RM.Hr_tot,label='Capacity Data', color = 'red')
axs[2,0].set_title('UCI/UTI Usage')
axs[2,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')

axs[2,1].set_xlim(0,days)
axs[2,1].plot(simulation1.t[:index1],simulation1.V[:index1],label='VMI Usage')
axs[2,1].plot(simulation2.t[:index2],simulation2.V[:index2],label='VMI Usage')
axs[2,1].plot(simulation3.t[:index3],simulation3.V[:index3],label='VMI Usage')
axs[2,1].scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
axs[2,1].scatter(RM.sochimi_tr,RM.Vr_tot,label='Capacity Data',color = 'red')
axs[2,1].set_title('UCI/UTI Usage')
axs[2,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


In [ ]:
print([Err])

## Detected Active Infected
Here we define a Detected infected population as follows:  

 Idet = Imi*det + Ise + Icr  

where det is the proportion of mild detections


**Aun me falta corregir el valor inicial, pues este lo considera para todos los infectados**

In [ ]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_det[:index],label='Infected')
plt.legend(loc=0)
plt.title('Detectable Active Infected')
plt.show()


## Detected Daily Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d_det[:index],label='Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.title('Detected Daily infected')
plt.show()

## Detected Accumulated Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac_det[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

# Analysis - Total infected

The following plots will show the simulation results with the real data

## Total Active Infected

In [ ]:
tr_index = np.searchsorted(RM.tr,days)

plt.scatter(RM.tr,RM.Ir,label='Real Data')

plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I[:index],label='Total Active Infected')
plt.legend(loc=0)
plt.title('Total Active Infected')
plt.show()


## Daily Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_d[:index],label='Total Daily new Infected')
plt.scatter(RM.I_d_r_tr,RM.I_d_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Acummulated Infected

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.I_ac[:index],label='Total Acummulated Infected')
plt.scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Real Data')
plt.legend(loc=0)
plt.show()

## Deaths

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.B[:index],label='Total Acummulated Deaths')
plt.scatter(RM.Br_tr,RM.Br,label='Real Data')
plt.legend(loc=0)
plt.show()

## UCI/UTI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.Hse[:index]+simulation.Hout[:index],label='UCI/UTI Beds')
plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.legend(loc=0)
plt.show()

## VMI

In [ ]:
plt.xlim(0,days)
plt.plot(simulation.t[:index],simulation.V[:index],label='VMI Usage')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.legend(loc=0)
plt.show()

# Objective Functions



In [ ]:
# Date until which we calculate the error (normally when the quarantine dynamic change)
t1_date = datetime(2020,8,15)


t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])

## Error:

In [ ]:
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])
print(Err)


In [ ]:
simulation.peak_date

In [ ]:
RM.initdate